![transformer_architecture](images/transformer_architecture.jpeg)

1. 源自encoder-decoder，在其他类型任务时，干掉decoder，只使用encoder；
2. 和RNN相比，最大的变化是并行化，为了并行化引入了：位置嵌入 (Positional Encoding) 
3. self-attention,自注意力机制
4. 
4. transformer
5. BERT
6. 基于BERT的各种nlp任务(文本分类，Qu相关性，中文分词，实体识别)

* https://wmathor.com/index.php/archives/1438/
* https://baijiahao.baidu.com/s?id=1651219987457222196&wfr=spider&for=pc
* https://zhuanlan.zhihu.com/p/59629215
* https://github.com/kuro7766/nlp_notebooks/blob/79866eab826e2ed15e41ef1ed2831415619e3c60/5-1.Transformer/Transformer.ipynb

In [4]:
import torch
from torch import nn, Tensor

# 1. Embedding

2. Positional Encoding

位置嵌入的维度为 [max_sequence_length, embedding_dimension], 和词向量的维度是相同的，因此，词嵌入和位置嵌入值可以相加。

位置嵌入参数是否可调整？

位置嵌入的生成，随着 embedding_dimension 序号增大，位置嵌入函数的周期变化越来越平缓

https://wmathor.com/index.php/archives/1453/

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

3. self-attention

参考这个：https://z3.ax1x.com/2021/04/20/c7wF9x.png#shadow

![QKV](https://z3.ax1x.com/2021/04/20/c7wF9x.png#shadow)
定义三个矩阵Wq,Wk,Wv，维度=embedding_dimension，大小vocab_size. 类比CNN里的卷积核？

![QKV_compute](https://z3.ax1x.com/2021/04/20/c7wk36.png#shadow)
查询向量*键矩阵，再softmax，再乘以值向量，再求和。

Q和K的点乘表示Q和K元素之间(每个元素都是向量)的相似程度.

为啥要除以dk，假设 Q 和 K 的均值为0，方差为1。它们的矩阵乘积将有均值为0，方差为dk，因此使用dk的平方根被用于缩放，因为，Q 和 K 的矩阵乘积的均值本应该为 0，方差本应该为1，这样可以获得更平缓的softmax。当维度很大时，点积结果会很大，会导致softmax的梯度很小。为了减轻这个影响，对点积进行缩放。

序列中的每个单词(token)和该序列中其余单词(token)进行 attention 计算

对应padding部分，填充0，softmax时需要计算，给无效区域加一个很大的负数偏置，经过softmax后值为0。
https://wmathor.com/index.php/archives/1557/
https://wmathor.com/index.php/archives/1540/

self-attention的可解释性：
透過計算Query和各個Key的相似性，得到每個Key對應Value的權重係數，權重係數代表訊息的重要性，亦即attention score；
Value則是對應的訊息，再對Value進行加權求和，得到最終的Attention/context vector。
https://github.com/nadavbh12/Character-Level-Language-Modeling-with-Deeper-Self-Attention-pytorch/blob/master/modules/attention.py

3.1 Multi-Head Attention
不同的关注点？
多头或者堆叠，Wq,Wk,Wv矩阵是共享的吗？


* https://pytorch.org/docs/master/generated/torch.nn.MultiheadAttention.html?highlight=attention#torch.nn.MultiheadAttention
* https://pytorch.org/docs/master/generated/torch.nn.TransformerDecoder.html?highlight=transformer#torch.nn.TransformerDecoder
* https://pytorch.org/docs/master/generated/torch.nn.TransformerEncoderLayer.html?highlight=attention
* https://pytorch.org/docs/master/generated/torch.nn.TransformerEncoder.html?highlight=transformer#torch.nn.TransformerEncoder
* https://pytorch.org/docs/master/generated/torch.nn.TransformerDecoderLayer.html?highlight=attention
* https://pytorch.org/docs/master/generated/torch.nn.Transformer.html?highlight=transformer#torch.nn.Transformer

In [7]:
#padding部分，有效区=0，无效区=-inf(很大的负数偏置)
def generate_square_subsequent_mask(self, sz: int) -> Tensor:
    r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
        Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
# 4. Add & Norm
# 词嵌入 + multi-head attention, 残差连接
# Layer Normalization， 和batch Normal不一样，按列求均值，再求方差.  训练和预测时有何不同？
# 每个 Encoder Block 中的 FeedForward 层权重都是共享的？

In [ ]:
# 5. Feed forward
# linear(relu(linear(x)))

In [ ]:
# Masked Self-Attention
# Mask 首先生成一个下三角全 0，上三角全为负无穷的矩阵，然后将其与 Scaled Scores 相加，
# 再做 softmax，就能将 - inf 变为 0，得到的这个矩阵即为每个字之间的权重

In [ ]:
# 堆叠的意义？
# 特定层是有独特的功能的，底层更偏向于关注语法，顶层更偏向于关注语义

In [ ]:
# Masked Encoder-Decoder Attention
# 和前面 Masked Self-Attention 很相似，结构也一摸一样，唯一不同的是这里的  
# K,V 为 Encoder 的输出， Q 为 Decoder 中 Masked Self-Attention 的输出